# データセットの分割数の違いを感じる
- 学習データで学習 → テストデータでテスト → 誤り率を計算する
- 上記を繰り返して、誤り率の平均と分散を算出する
- CVの分割数 と ホールドアウト割合 を変えて算出する

# 実装のメモ

### アルゴリズムの流れ

1. データセットを取得

2. データセットを学習データと教師データに分割

3. 学習データを元にパラメータを学習

4. 教師データで誤り率を算出

5. 2〜4 を繰り返して、誤り率の標本データを取得し、平均分散を算出

6. 5 を CVの分割数 と ホールドアウトの割合 を変えて算出する


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
%matplotlib inline

/Users/yohei.moriya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yohei.moriya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yohei.moriya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yohei.moriya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yohei.moriya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yoh

In [2]:
def get_data() -> pd.DataFrame:
    '''
    データセットを作成します
    '''
    origin_data = load_breast_cancer()
    df_origin = pd.DataFrame(origin_data.data, columns=origin_data.feature_names)
    df_origin['target'] = origin_data.target
    return df_origin

# test
df_origin = get_data()
df_origin.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
from abc import ABCMeta, abstractmethod


class Spliter(metaclass=ABCMeta):
    '''
    データの分割器です。
    '''
    def __init__(self, df, num):
        '''
        分割器を初期化します。
        '''
        self.df = df
        self.num = num

    @abstractmethod
    def run(self):
        '''
        データの分割を実行します。
        
        :return TrainDataSet, TestDataSet
        '''
        raise NotImplementedError()
        
    def to_str(self):
        return self.__class__.__name__ + '#' + str(self.num)


class FOSpliter(Spliter): 
    '''
    Fold Out法でデータを分割する分割器です。
    '''
    def __init__(self, df, num):
        '''
        :param df 分割したいデータ
        :param num データを分割するレート
        '''
        super().__init__(df, num)
    
    def run(self) -> pd.DataFrame:
        df_train = self.df.sample(frac=self.num)
        df_test = self.df.drop(df_train.index)
        yield df_train, df_test


class CVSpliter(Spliter):
    '''
    Cross Varidation法でデータを分割する分割器です。
    '''
    def __init__(self, df, num):
        '''
        :param df 分割したいデータ
        :param num データの分割する個数
        '''
        super().__init__(df, num)
        
    def run(self) -> pd.DataFrame:
        fold = KFold(n_splits=self.num, shuffle=True, random_state=1)
        for ids_train, ids_test in fold.split(self.df):
            df_train, df_test = self.df.iloc[ids_train], self.df.iloc[ids_test]
            yield df_train, df_test

#  test
df = get_data()
spliter_fo = FOSpliter(df=df, num=0.5)
spliter_cv = CVSpliter(df=df, num=4)
tasks = [spliter_fo, spliter_cv]
for task in tasks:
    print(task.to_str())
    for df_train, df_test in task.run():
        display(df_train.head())
        display(df_test.head())


FOSpliter#0.5


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
384,13.280,13.72,85.79,541.8,0.08363,0.08575,0.050770,0.028640,0.1617,0.05594,...,17.37,96.59,623.7,0.11660,0.26850,0.28660,0.09173,0.2736,0.07320,1
326,14.110,12.88,90.03,616.5,0.09309,0.05306,0.017650,0.027330,0.1373,0.05700,...,18.00,98.40,749.9,0.12810,0.11090,0.05307,0.05890,0.2100,0.07083,1
48,12.050,14.63,78.04,449.3,0.10310,0.09092,0.065920,0.027490,0.1675,0.06043,...,20.70,89.88,582.6,0.14940,0.21560,0.30500,0.06548,0.2747,0.08301,1
539,7.691,25.44,48.34,170.4,0.08668,0.11990,0.092520,0.013640,0.2037,0.07751,...,31.89,54.49,223.6,0.15960,0.30640,0.33930,0.05000,0.2790,0.10660,1
307,9.000,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,0.1788,0.06833,...,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804,1


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
3,11.42,20.38,77.58,386.1,0.14250,0.2839,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
5,12.45,15.70,82.57,477.1,0.12780,0.1700,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0
6,18.25,19.98,119.60,1040.0,0.09463,0.1090,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
7,13.71,20.83,90.20,577.9,0.11890,0.1645,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,0


CVSpliter#4


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
6,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
7,13.71,20.83,90.2,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.6,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,0
8,13.00,21.82,87.5,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.2,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,0


...

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
6,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
11,15.78,17.89,103.6,781.0,0.09710,0.12920,0.09954,0.06606,0.1842,0.06082,...,27.28,136.5,1299.0,0.1396,0.5609,0.3965,0.1810,0.3792,0.10480,0
12,19.17,24.80,132.4,1123.0,0.09740,0.24580,0.20650,0.11180,0.2397,0.07800,...,29.94,151.7,1332.0,0.1037,0.3903,0.3639,0.1767,0.3176,0.10230,0
13,15.85,23.95,103.7,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,0.05338,...,27.66,112.0,876.5,0.1131,0.1924,0.2322,0.1119,0.2809,0.06287,0


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
8,13.000,21.82,87.50,519.8,0.1273,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.53900,0.20600,0.4378,0.10720,0
21,9.504,12.44,60.34,273.9,0.1024,0.06492,0.02956,0.02076,0.1815,0.06905,...,15.66,65.13,314.9,0.1324,0.1148,0.08867,0.06227,0.2450,0.07773,1
24,16.650,21.38,110.00,904.6,0.1121,0.14570,0.15250,0.09170,0.1995,0.06330,...,31.56,177.00,2215.0,0.1805,0.3578,0.46950,0.20950,0.3613,0.09564,0
28,15.300,25.27,102.40,732.4,0.1082,0.16970,0.16830,0.08751,0.1926,0.06540,...,36.71,149.30,1269.0,0.1641,0.6110,0.63350,0.20240,0.4027,0.09876,0
32,17.020,23.98,112.80,899.3,0.1197,0.14960,0.24170,0.12030,0.2248,0.06382,...,32.09,136.10,1344.0,0.1634,0.3559,0.55880,0.18470,0.3530,0.08482,0


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758,0
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510,0
10,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,0.05697,...,33.88,123.80,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452,0
15,14.54,27.54,96.73,658.8,0.11390,0.15950,0.16390,0.07364,0.2303,0.07077,...,37.13,124.10,943.2,0.1678,0.6577,0.7026,0.17120,0.4218,0.13410,0
20,13.08,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.03110,0.1967,0.06811,...,20.49,96.09,630.5,0.1312,0.2776,0.1890,0.07283,0.3184,0.08183,1


In [5]:
def col_error(df_train: pd.DataFrame, df_test: pd.DataFrame):
    '''
    ロジスティック回帰を用いてテストデータの誤り率を計算します。
    '''
    df_train_x = df_train.drop('target', axis=1)
    df_train_y = df_train['target']
    lr = LogisticRegression()
    lr.fit(df_train_x, df_train_y)
    
    df_test_x = df_test.drop('target', axis=1)
    df_test_y = df_test['target']
    score = lr.score(df_test_x, df_test_y)
    
    return 1- score

# test
df = get_data()
print(col_error(df, df))

0.040421792618629215


In [6]:
class Aggregator():
    '''
    情報を貯蓄して解析する集積器です。
    内部でキーに対応したリストを保持しています。
    '''
    def __init__(self):
        self.values = {}

    def savings(self, key, value):
        '''
        値をキーに対応したリストに保存します。
        '''
        if key not in self.values:
            self.values[key] = []
        self.values[key].append(value)
        pass
    
    def total(self):
        '''
        貯蓄した値を集計します。
        '''
        dict_result = {}
        for key in self.values:
            info = {}
            info['mean'] = np.array(self.values[key]).mean()
            info['var'] = np.array(self.values[key]).var()
            dict_result[key] = info
        return pd.DataFrame.from_dict(dict_result).T
    
    def destory(self):
        '''
        貯蔵した値を破棄します。
        '''
        self.values = {}

memory = Aggregator()
memory.savings('key1', 1)
memory.savings('key1', 2)
memory.savings('key2', 3)
print(memory.values)

{'key1': [1, 2], 'key2': [3]}


In [16]:
def run_task(memory, task):
    error = []
    for df_train, df_test in task.run():
        error.append(col_error(df_train, df_test))
    memory.savings(task.to_str(), np.mean(error))
    return memory

def run_tasks(memory, tasks):
    for task in tasks:
        memory = run_task(memory, task)
    return memory

def main():
    memory = Aggregator()
    for i in range(10):
        df_origin = get_data()
        tasks_fo = [FOSpliter(df=df_origin, num=round(num*0.1, 1)) for num in range(1, 10)]
        tasks_cv = [CVSpliter(df=df_origin, num=(num)) for num in range(2, 11)]
        tasks = tasks_fo + tasks_cv
        memory = run_tasks(memory, tasks)
    display(memory.total())

main()

,mean,var
FOSpliter#0.1,0.083203,2.900696e-04
FOSpliter#0.2,0.060879,1.179085e-04
FOSpliter#0.3,0.054271,8.989672e-05
FOSpliter#0.4,0.055718,8.943852e-05
FOSpliter#0.5,0.052982,7.743921e-05
FOSpliter#0.6,0.051754,1.454294e-04
FOSpliter#0.7,0.052632,8.891625e-05
FOSpliter#0.8,0.056140,4.493690e-04
FOSpliter#0.9,0.050877,1.508156e-04
CVSpliter#2,0.052687,0.000000e+00
